# Summary

**Airbnb listings in Melbourne city**

Detailed listings data for Melbourne
The data behind the Inside Airbnb site is sourced from publicly available information from the Airbnb site.

[Dataset source](http://insideairbnb.com/get-the-data.html)

**Stats**

* 22,909 listings
* 14,849 unique hosts
* 14,599 listings for "whole apartment"
* 9,090 unique hosts for "whole apartment"


* 30 neighbourhoods
* 485 cities
* 538 streets


* bathrooms: min 0.5, max 14, median 1
* bedrooms: min 0, max 16, median 2
* price: expressed in dollars min 0, max 999, avg. 180

# Goals

This analysis will explore the dataset provided by [insideairbnb.com](http://insideairbnb.com/index.html).

The goal here is to get this cleaned up a little and into Elasticsearch where we'll explore further.

## Thanks to the Elastic Machine Learning team for the work this example is based on :)!

In [5]:
import pandas as pd
import numpy as np
import json
import csv
from elasticsearch import helpers, Elasticsearch
import requests
import re
import os

In [6]:
df = pd.read_csv("listings.csv")

/Users/emanuil/elastic/talks/data-science-intro-to-es-kibana-TALK/dsenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (43,61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


You can ignore the DType warning the line above may produce.

In [7]:
df = df[["id",
"host_id",
"street",
"neighbourhood_cleansed",
"city",
"state",
"zipcode",
"country",
"latitude",
"longitude",
"property_type",
"room_type",
"bathrooms",
"bedrooms",
"square_feet",
"price",
"number_of_reviews"]]

In [8]:
df.head()

,id,host_id,street,neighbourhood_cleansed,city,state,zipcode,country,latitude,longitude,property_type,room_type,bathrooms,bedrooms,square_feet,price,number_of_reviews
0,9835,33057,"Bulleen, VIC, Australia",Manningham,Bulleen,VIC,3105,Australia,-37.77268,145.09213,House,Private room,1.0,1.0,NaN,$60.00,4
1,10803,38901,"Brunswick East, VIC, Australia",Moreland,Brunswick East,VIC,3057,Australia,-37.76651,144.98074,Apartment,Private room,1.0,1.0,226.0,$35.00,129
2,12936,50121,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,Australia,-37.85976,144.97737,Apartment,Entire home/apt,1.0,1.0,0.0,$159.00,30
3,15246,59786,"Thornbury, VIC, Australia",Darebin,Thornbury,VIC,3071,Australia,-37.75897,144.98923,House,Private room,1.5,1.0,0.0,$49.00,29
4,16760,65090,"St Kilda East, VIC, Australia",Port Phillip,St Kilda East,VIC,3183,Australia,-37.86453,144.99224,House,Private room,1.0,1.0,NaN,$68.00,75


## Cleaning and pre-processing

**Set the number of bedrooms to 1 in case it's a NaN 
and the number of bathroom to 1 if it's either a NaN or 0**

In [9]:
df["bedrooms"].replace(np.nan, 1, inplace=True)
df["bathrooms"].replace(np.nan, 1, inplace=True)
df["bathrooms"].replace(0, 1, inplace=True)

**Cleaning the `price` field and changing its type to float**

In [10]:
reg = "\$([0-9]*)(\.|\,){1,1}.*"
df["price"] = df["price"].apply(lambda x: re.search(reg, x).group(1))
df["price"] = df["price"].astype("float")

## Stats

In [11]:
print(f"Number of listings: {len(df)}")
print(f"Number of hosts: {len(df.groupby('host_id'))}")

Number of listings: 24194
Number of hosts: 15284


**Now we'll focus on "whole apartment" listings only, filtering out the ones with shared/private bedrooms**

In [12]:
entire_place = df[(df["room_type"]=="Entire home/apt")]
entire_place = entire_place.drop('room_type', axis=1)

In [13]:
print(f"Number of listings: {len(entire_place)}")
print(f"Number of hosts: {len(entire_place.groupby('host_id'))}")

Number of listings: 14868
Number of hosts: 9177


**The number of states is > 1**
**Looking at the data we can see that the field contains different variation of 'Victoria' and other terms. Let's replace the state value for each entry with the unique term'Victoria'**

In [14]:
print(f"Number of unique states: {len(entire_place['state'].unique())}")
print(entire_place["state"].unique())

Number of unique states: 11
['VIC' 'Victoria' 'Vic' 'Melbourne' nan 'Australia' 'victoria' 'VICTORIA'
 '维多利亚' 'vic' 'VIV']


In [15]:
entire_place["state"] = "Victoria"

In [16]:
print(f"Number of unique cities: {len(entire_place['city'].unique())}")
print(f"Number of unique neighbourhood: {len(entire_place['neighbourhood_cleansed'].unique())}")
print(f"Number of unique street: {len(entire_place['street'].unique())}")

Number of unique cities: 494
Number of unique neighbourhood: 30
Number of unique street: 542


### Recording the cleaned data to disk

In [17]:
cleaned_file = f"cleaned_melbourne_listings.csv"

In [19]:
entire_place.to_csv(cleaned_file, header=False, index=False)

## Putting the data in Elasticsearch - mappings and indexing

In [20]:
mappings = {
    "settings" : {
        "number_of_shards" : 1
    },
    "mappings" : {   
            "properties" : {
                "listing_id" : {"type" : "keyword"},
                "host_id" : {"type" : "keyword"},
                "street" : {"type" : "keyword"},
                "neighbourhood" : {"type" : "keyword"},
                "city" : {"type" : "keyword"},
                "state" : {"type" : "keyword"},
                "zipcode" : {"type" : "keyword"},
                "country" : {"type" : "keyword"},
                "location" : {"type": "geo_point"},
                "property_type" : {"type" : "keyword"},
                "bathrooms" : {"type" : "float"},
                "bedrooms" : {"type" : "float"},
                "price" : {"type" : "float"},
                "number_of_reviews" : {"type" : "integer"}
                
            }
    }
}

In [21]:
INDEX_NAME = 'airbnb_melbourne'

In [22]:
def get_data():
    with open(cleaned_file, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file,
                                    fieldnames=["listing_id", "host_id", "street",
                                               "neighbourhood", "city", "state",
                                               "zipcode", "country", "latitude",
                                               "longitude", "property_type", 
                                               "bathrooms", "bedrooms", "square_feet",
                                               "price", "number_of_reviews"])
        for i in csv_reader:
            i["location"] = {"lat": i["latitude"], "lon": i["longitude"]}
            del i["latitude"]
            del i["longitude"]
            yield {
                "_index": INDEX_NAME,
                "_source": i
            }

### Use Elastic Cloud

In [23]:
es = Elasticsearch(
    [os.environ['ELASTIC_CLOUD_ES_ENDPOINT']],
    http_auth=('elastic', os.environ['ELASTIC_CLOUD_PASSWORD']),
    scheme="https",
    port=443,
)

In [24]:
es.indices.delete(INDEX_NAME, ignore=[400, 404])  # to reset

{'error': {'root_cause': [{'type': 'index_not_found_exception',
    'reason': 'no such index [airbnb_melbourne]',
    'resource.type': 'index_or_alias',
    'resource.id': 'airbnb_melbourne',
    'index_uuid': '_na_',
    'index': 'airbnb_melbourne'}],
  'type': 'index_not_found_exception',
  'reason': 'no such index [airbnb_melbourne]',
  'resource.type': 'index_or_alias',
  'resource.id': 'airbnb_melbourne',
  'index_uuid': '_na_',
  'index': 'airbnb_melbourne'},
 'status': 404}

In [25]:
es.indices.create(INDEX_NAME, mappings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'airbnb_melbourne'}

In [26]:
helpers.bulk(es, get_data())
# now show the creation of the Kibana Index Pattern and explain what it's for

(14868, [])

# Retrieve data from Elasticsearch into pandas

In [27]:
q = {"query": {"match_all": {}}, "size": 1000}
results = helpers.scan(es, query=q, index="stkilda")

In [28]:
processed = []
for r in results:
    r['_source']['price_avg'] = r['_source']['price']['avg']
    del r['_source']['price']
    processed.append(r['_source'])

In [29]:
df = pd.DataFrame.from_dict(processed)

In [30]:
df

,property_type,price_avg
0,Apartment,150.184685
1,Condominium,128.750000
2,Cottage,150.000000
3,Guest suite,145.000000
4,House,247.263158
5,Loft,122.750000
6,Serviced apartment,121.000000
7,Townhouse,230.142857
8,Villa,950.000000
